In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

from src.hrv_epatch.io.tdms import load_tdms_from_path

In [ ]:
OUT_DIR = Path(r"E:\Speciale - Results\Datastruct")

df_rec = pd.read_csv(
    OUT_DIR / "recordings_index.csv",
    parse_dates=["recording_start", "recording_end"],
)

df_evt = pd.read_csv(
    OUT_DIR / "seizure_events.csv",
    parse_dates=["absolute_start", "absolute_end"],
)
# Sanity check
print(df_rec.head())
print(df_evt.head())


# Create copies to avoid unintentional modifications
df_rec_sum = df_rec.copy()
df_evt_sum = df_evt.copy()

   recording_uid  patient_id enrollment_id  recording_id  \
0              0           1           NaN             1   
1              1           1           NaN             2   
2              2           2           NaN             1   
3              3           3           NaN             1   
4              4           3           NaN             2   

                                           tdms_path annotation_source  \
0  E:\ML algoritme tl anfaldsdetektion vha HRV\eP...     patient 1.xls   
1  E:\ML algoritme tl anfaldsdetektion vha HRV\eP...     patient 1.xls   
2  E:\ML algoritme tl anfaldsdetektion vha HRV\eP...     patient 2.xls   
3  E:\ML algoritme tl anfaldsdetektion vha HRV\eP...     patient 3.xls   
4  E:\ML algoritme tl anfaldsdetektion vha HRV\eP...     patient 3.xls   

      recording_start                 recording_end  rec_duration_s     fs  
0 2016-02-22 11:04:14 2016-02-24 16:09:49.750000000   191135.750000  512.0  
1 2016-02-24 16:15:00 2016-02-26 09:00:0

In [ ]:
def build_annotations_dict_from_evt_and_rec(df_evt: pd.DataFrame, df_rec: pd.DataFrame):
    """
    Bygger annotations_dict ud fra:
      - df_evt: seizure_events (t0/t1 i sekunder fra recording-start)
      - df_rec: recordings_index (annotation_source per recording)

    Returnerer:
      dict[recording_uid] -> list af {"t0", "t1", "label", "kind"}
    """

    # lav opslag recording_uid -> annotation_source
    rec_src = (
        df_rec[["recording_uid", "annotation_source"]]
        .set_index("recording_uid")["annotation_source"]
        .to_dict()
    )

    annotations_dict = {}

    for _, r in df_evt.iterrows():
        uid = int(r["recording_uid"])
        t0 = float(r["t0"])
        t1 = float(r["t1"])
        seizure_id = int(r["seizure_id"])

        label = f"Seizure {seizure_id}"

        # hent annotation_source for den recording (kan være NaN)
        kind = rec_src.get(uid, "generic")
        if pd.isna(kind):
            kind = "generic"

        ann = {
            "t0": t0,
            "t1": t1,
            "label": label,
            "kind": str(kind),
        }

        annotations_dict.setdefault(uid, []).append(ann)

    return annotations_dict

In [ ]:
def _color_and_yspan_from_kind(kind: str):
    """
    Map annotation_source -> (color, ymin, ymax).
    Tilpas match til dine reelle annotation_source-værdier.
    """
    k = str(kind).lower()

    # defaults
    color = "tab:blue"
    ymin, ymax = 0.0, 1.0

    if "video" in k or "veeg" in k:
        color = "tab:red"
        ymin, ymax = 0.5, 1.0  # øverste halvdel
    elif "clinic" in k or "klin" in k:
        color = "tab:green"
        ymin, ymax = 0.0, 0.5  # nederste halvdel
    else:
        color = "tab:blue"
        ymin, ymax = 0.0, 1.0

    return color, ymin, ymax


def plot_ecg_segment_with_spans(
    tdms_path,
    fs,
    t_start_s,
    t_window_s=30.0,
    channel_hint="EKG",
    interval_annotations=None,
    title=None,
):
    sig, _ = load_tdms_from_path(tdms_path, channel_hint=channel_hint)
    sig = np.asarray(sig)
    if sig.ndim == 2:
        sig = sig[:, 0]

    n = sig.size
    start_idx = int(t_start_s * fs)
    end_idx = int((t_start_s + t_window_s) * fs)
    start_idx = max(0, min(start_idx, n - 1))
    end_idx = max(start_idx + 1, min(end_idx, n))

    seg = sig[start_idx:end_idx]
    t = np.arange(seg.size) / fs + t_start_s

    fig, ax = plt.subplots(figsize=(14, 4))

    # baggrundsspans
    if interval_annotations:
        for ann in interval_annotations:
            t0 = float(ann["t0"])
            t1 = float(ann["t1"])

            if t1 < t_start_s or t0 > t_start_s + t_window_s:
                continue

            t0_clip = max(t0, t_start_s)
            t1_clip = min(t1, t_start_s + t_window_s)

            color, ymin, ymax = _color_and_yspan_from_kind(ann.get("kind", ""))

            ax.axvspan(
                t0_clip,
                t1_clip,
                ymin=ymin,
                ymax=ymax,
                facecolor=color,
                alpha=0.2,
                zorder=0,
            )

            ax.text(
                (t0_clip + t1_clip) / 2,
                0.99,
                ann.get("label", ""),
                transform=ax.get_xaxis_transform(),
                ha="center",
                va="top",
                fontsize=8,
                color=color,
            )

    # selve ECG
    ax.plot(t, seg, linewidth=0.8, color="k", zorder=1)
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("ECG [ADC]")
    if title:
        ax.set_title(title)
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    plt.show()



def plot_ecg_segment_with_annotations(
    tdms_path,
    fs,
    t_start_s,
    t_window_s=30.0,
    channel_hint="EKG",
    annotations=None,
    title=None,
):
    """
    Plotter et udsnit af ECG med annotationer.
    
    annotations: liste af dicts:
        [{"time_s": ..., "label": "Video-EEG", "color": "r"},
         {"time_s": ..., "label": "Clinical", "color": "g"}, ...]
    """
    sig, _ = load_tdms_from_path(tdms_path, channel_hint=channel_hint)
    sig = np.asarray(sig)
    if sig.ndim == 2:
        sig = sig[:, 0]

    n = sig.size
    start_idx = int(t_start_s * fs)
    end_idx = int((t_start_s + t_window_s) * fs)
    start_idx = max(0, min(start_idx, n - 1))
    end_idx = max(start_idx + 1, min(end_idx, n))

    seg = sig[start_idx:end_idx]
    t = np.arange(seg.size) / fs + t_start_s

    plt.figure(figsize=(14, 4))
    plt.plot(t, seg, linewidth=0.8)

    if annotations:
        for ann in annotations:
            t_ann = ann["time_s"]
            if t_start_s <= t_ann <= t_start_s + t_window_s:
                plt.axvline(t_ann, color=ann.get("color", "r"), linestyle="--", alpha=0.8)
                plt.text(
                    t_ann,
                    np.max(seg),
                    ann.get("label", ""),
                    rotation=90,
                    va="bottom",
                    ha="right",
                    fontsize=8,
                )

    plt.xlabel("Time [s]")
    plt.ylabel("ECG [ADC]")
    if title:
        plt.title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

def launch_ecg_viewer(df_rec, annotations_dict, channel_hint="EKG"):
    """
    df_rec: dit recording-index (med tdms_path, fs, patient_id, recording_id, recording_uid)
    annotations_dict: dict mapping recording_uid -> list of annotations
        annotations_dict[rec_uid] = [
            {"time_s": ..., "label": "Video-EEG", "color": "r"},
            {"time_s": ..., "label": "Clinical", "color": "g"},
            ...
        ]
    """

    rec_options = {
        f"P{int(r.patient_id):02d}_R{int(r.recording_id):02d} (uid={r.recording_uid})": r
        for _, r in df_rec.iterrows()
    }

    rec_dropdown = widgets.Dropdown(
        options=list(rec_options.keys()),
        description="Recording:",
        layout=widgets.Layout(width="60%"),
    )

    t_start_slider = widgets.FloatSlider(
        value=0.0,
        min=0.0,
        max=3600.0,  # kan evt. opdateres dynamisk
        step=10.0,
        description="t_start [s]",
        continuous_update=False,
        layout=widgets.Layout(width="60%"),
    )

    t_window_slider = widgets.FloatSlider(
        value=30.0,
        min=5.0,
        max=300.0,
        step=5.0,
        description="window [s]",
        continuous_update=False,
        layout=widgets.Layout(width="60%"),
    )

    def _update_max_time(change):
        # opdatér max for t_start når recording skifter
        key = rec_dropdown.value
        r = rec_options[key]
        fs = float(r.fs)
        n_samples = int(r.n_samples)
        rec_dur = n_samples / fs
        t_start_slider.max = max(10.0, rec_dur - 10.0)

    rec_dropdown.observe(_update_max_time, names="value")
    _update_max_time(None)

    @interact(
        rec_key=rec_dropdown,
        t_start_s=t_start_slider,
        t_window_s=t_window_slider,
    )
    def _view(rec_key, t_start_s, t_window_s):
        r = rec_options[rec_key]
        tdms_path = r.tdms_path
        fs = float(r.fs)
        rec_uid = r.recording_uid

        anns = annotations_dict.get(rec_uid, [])

        title = f"Patient {int(r.patient_id)}, recording {int(r.recording_id)} (uid={rec_uid})"

        plot_ecg_segment_with_annotations(
            tdms_path=tdms_path,
            fs=fs,
            t_start_s=t_start_s,
            t_window_s=t_window_s,
            channel_hint=channel_hint,
            annotations=anns,
            title=title,
        )


In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

def launch_ecg_viewer_with_spans(df_rec, annotations_dict, channel_hint="EKG"):
    rec_options = {
        f"P{int(r.patient_id):02d}_R{int(r.recording_id):02d} (uid={r.recording_uid})": r
        for _, r in df_rec.iterrows()
    }

    rec_dropdown = widgets.Dropdown(
        options=list(rec_options.keys()),
        description="Recording:",
        layout=widgets.Layout(width="60%"),
    )

    t_start_slider = widgets.FloatSlider(
        value=0.0,
        min=0.0,
        max=3600.0,
        step=10.0,
        description="t_start [s]",
        continuous_update=False,
        layout=widgets.Layout(width="60%"),
    )

    t_window_slider = widgets.FloatSlider(
        value=30.0,
        min=5.0,
        max=300.0,
        step=5.0,
        description="window [s]",
        continuous_update=False,
        layout=widgets.Layout(width="60%"),
    )

    def _update_max_time(change):
        key = rec_dropdown.value
        r = rec_options[key]
        fs = float(r.fs)
        # rec_df har rec_duration_s, så vi kan bruge den:
        rec_dur = float(r.rec_duration_s)
        t_start_slider.max = max(10.0, rec_dur - 10.0)

    rec_dropdown.observe(_update_max_time, names="value")
    _update_max_time(None)

    @interact(
        rec_key=rec_dropdown,
        t_start_s=t_start_slider,
        t_window_s=t_window_slider,
    )
    def _view(rec_key, t_start_s, t_window_s):
        r = rec_options[rec_key]
        tdms_path = r.tdms_path
        fs = float(r.fs)
        rec_uid = r.recording_uid

        anns = annotations_dict.get(rec_uid, [])

        title = f"Patient {int(r.patient_id)}, recording {int(r.recording_id)} (uid={rec_uid})"

        plot_ecg_segment_with_spans(
            tdms_path=tdms_path,
            fs=fs,
            t_start_s=t_start_s,
            t_window_s=t_window_s,
            channel_hint=channel_hint,
            interval_annotations=anns,
            title=title,
        )


In [ ]:
df_rec = pd.read_csv("recordings_index.csv")
df_evt = pd.read_csv("seizure_events.csv")

annotations_dict = build_annotations_dict_from_evt_and_rec(df_evt, df_rec)

launch_ecg_viewer_with_spans(df_rec, annotations_dict)


AttributeError: 'Series' object has no attribute 'n_samples'